In [28]:
# v2 imports temperature records for a date range across 1 year using the datetime library

import requests
import pandas as pd
from datetime import datetime, timedelta

# Step 1: Define the API endpoint
url = "https://api.data.gov.sg/v1/environment/air-temperature"

# Define the start and end dates for the range
start_date = datetime(2023, 11, 19)  # Start date: 2023-11-01
end_date = datetime(2024, 11, 20)  # End date: 2023-11-10

# List to hold all the data
all_data = []

# Loop through the date range and request data for each day
current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime("%Y-%m-%dT%H:%M:%S")  # Format date for the API request
    params = {"date_time": date_str}  # Use the 'date_time' parameter for the API request
    
    # Send the GET request to the API
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        json_data = response.json()
        
        # Extract and flatten the "readings" for each station
        items = json_data["items"]
        readings = [reading for item in items for reading in item["readings"]]
        
        # Create a DataFrame from the readings
        df = pd.DataFrame(readings)
        
        # Merge with station metadata
        stations = pd.DataFrame(json_data["metadata"]["stations"])
        df = df.merge(stations, left_on="station_id", right_on="id", how="left")
        
        # Drop the 'id' column to clean up the DataFrame
        df.drop(columns=["id"], inplace=True)
        
        # Add the current date to the DataFrame for easy tracking
        df["date"] = date_str
        
        # Append the data for this day to the all_data list
        all_data.append(df)
    else:
        print(f"Failed to fetch data for {date_str} with status code {response.status_code}")
    
    # Increment the current date by one day
    current_date += timedelta(days=1)

# If any data was fetched, concatenate all data frames into one
if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    
    # Convert 'date' column to datetime format
    final_df["date"] = pd.to_datetime(final_df["date"])

    # Preview the final DataFrame
    print(final_df.head())
    print(final_df.info())
    display(final_df)
    
    # Optional: Save the data to a CSV file
    final_df.to_csv('airtempacrosssg.csv', index=False)
else:
    print("No data was fetched during the specified date range.")

  station_id  value device_id                 name  \
0       S109   27.2      S109  Ang Mo Kio Avenue 5   
1       S117   27.6      S117          Banyan Road   
2        S50   27.2       S50        Clementi Road   
3       S107   28.1      S107   East Coast Parkway   
4        S43   27.8       S43       Kim Chuan Road   

                                      location       date  
0  {'latitude': 1.3764, 'longitude': 103.8492} 2023-11-19  
1    {'latitude': 1.256, 'longitude': 103.679} 2023-11-19  
2  {'latitude': 1.3337, 'longitude': 103.7768} 2023-11-19  
3  {'latitude': 1.3135, 'longitude': 103.9625} 2023-11-19  
4  {'latitude': 1.3399, 'longitude': 103.8878} 2023-11-19  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4821 entries, 0 to 4820
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   station_id  4821 non-null   object        
 1   value       4821 non-null   float64       
 2   device_id   

,station_id,value,device_id,name,location,date
0,S109,27.2,S109,Ang Mo Kio Avenue 5,"{'latitude': 1.3764, 'longitude': 103.8492}",2023-11-19
1,S117,27.6,S117,Banyan Road,"{'latitude': 1.256, 'longitude': 103.679}",2023-11-19
2,S50,27.2,S50,Clementi Road,"{'latitude': 1.3337, 'longitude': 103.7768}",2023-11-19
3,S107,28.1,S107,East Coast Parkway,"{'latitude': 1.3135, 'longitude': 103.9625}",2023-11-19
4,S43,27.8,S43,Kim Chuan Road,"{'latitude': 1.3399, 'longitude': 103.8878}",2023-11-19
...,...,...,...,...,...,...
4816,S60,26.9,S60,Sentosa,"{'latitude': 1.25, 'longitude': 103.8279}",2024-11-20
4817,S115,26.6,S115,Tuas South Avenue 3,"{'latitude': 1.29377, 'longitude': 103.61843}",2024-11-20
4818,S24,25.8,S24,Upper Changi Road North,"{'latitude': 1.3678, 'longitude': 103.9826}",2024-11-20
4819,S116,26.7,S116,West Coast Highway,"{'latitude': 1.281, 'longitude': 103.754}",2024-11-20
